# Taiwan Universities - Which is the Best for Living?

This report is intented to give a full review on the convenience of each university. Though it may be somewhat absurd only choosing universities based on their convenience, it is still something worth noticing, especially for those who haven't actually been to a specific university before. 

## Data Processing for Taiwanese Universities

In [87]:
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm

rdf = pd.read_csv('ulistaddress_109-12-30.csv')

In [88]:
rdf.head()

,序號,學校代碼,公私立,體制,學校名稱,學校英文名稱,職稱,姓名,縣市別,第三級行政區,郵遞區號,學校地址,學校總機,學校傳真,網址
0,1,0001,公立,一般大學,國立政治大學,National Chengchi Universerity,校長,郭明政,臺北市,文山區,11605,臺北市文山區指南路2段64號,02-2939-3091,02-2937-9611,http://www.nccu.edu.tw
1,2,0002,公立,一般大學,國立清華大學,National Tsing Hua University,校長,賀陳弘,新竹市,新竹市,30013,新竹市光復路2段101號,03-571-5131,03-572-4038,http://www.nthu.edu.tw
2,3,0003,公立,一般大學,國立臺灣大學,NATIONAL TAIWAN UNIVERSITY,校長,管中閔,臺北市,大安區,10617,臺北市大安區羅斯福路4段1號,02-3366-3366,02-2362-7651,http://www.ntu.edu.tw
3,4,0004,公立,一般大學,國立臺灣師範大學,National Taiwan Normal University,校長,吳正己,臺北市,大安區,10610,臺北市大安區和平東路1段162號,02-7749-1111,無,http://www.ntnu.edu.tw
4,5,0005,公立,一般大學,國立成功大學,National Cheng Kung University,校長,蘇慧貞,臺南市,東 區,70101,臺南市東區大學路1號,06-275-7575,06-276-6462,http://www.ncku.edu.tw


In [89]:
# 序號,"學校代碼",公私立,體制,學校名稱,學校英文名稱,職稱,姓名,縣市別,"第三級行政區","郵遞區號",學校地址,學校總機,學校傳真,網址
pdf = rdf.rename(columns={'序號': 'Serial Number', '學校代碼': 'ID', '公私立': 'Public/Private', '體制': 'System', '學校名稱': 'Name', '學校英文名稱': 'Eng. Name', '郵遞區號':'Postal Code', '學校總機': 'Tel','學校傳真': 'Fax','網址': 'URL'})
pdf['Public/Private'].replace(to_replace=['公立','私立'], value=['Public', 'Private'],inplace=True)
selected = ['Serial Number',
 'ID',
 'Public/Private',
 'System',
 'Name',
 'Eng. Name',
 'Postal Code',
 '縣市別',
 '第三級行政區',
 '學校地址']

pdf = pdf[selected]
pdf.head()

,Serial Number,ID,Public/Private,System,Name,Eng. Name,Postal Code,縣市別,第三級行政區,學校地址
0,1,0001,Public,一般大學,國立政治大學,National Chengchi Universerity,11605,臺北市,文山區,臺北市文山區指南路2段64號
1,2,0002,Public,一般大學,國立清華大學,National Tsing Hua University,30013,新竹市,新竹市,新竹市光復路2段101號
2,3,0003,Public,一般大學,國立臺灣大學,NATIONAL TAIWAN UNIVERSITY,10617,臺北市,大安區,臺北市大安區羅斯福路4段1號
3,4,0004,Public,一般大學,國立臺灣師範大學,National Taiwan Normal University,10610,臺北市,大安區,臺北市大安區和平東路1段162號
4,5,0005,Public,一般大學,國立成功大學,National Cheng Kung University,70101,臺南市,東 區,臺南市東區大學路1號


In [90]:
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values
import requests
import json
import geocoder

In [91]:
# Credentials HERE api
APP_ID = 'wSPV4qOIeO14FPcYiHWE'
APP_CODE = 'Yk_h4AIMIQ8nLuwMP_DUvg'

In [92]:
# define the dataframe columns
column_names = ['Serial Number',
 'ID',
 'Public/Private',
 'System',
 'Name',
 'Eng. Name',
 'Postal Code', 
 'Latitude', 
 'Longitude', 
 'Address',
 'City',
 'County',
 'Neighborhood'] 

# instantiate the dataframe
updf = pd.DataFrame(columns=column_names)
updf

,Serial Number,ID,Public/Private,System,Name,Eng. Name,Postal Code,Latitude,Longitude,Address,City,County,Neighborhood


In [94]:
for data in tqdm(pdf.to_dict(orient='records')):
    """
    ['Serial Number',
     'ID',
     'Public/Private',
     'System',
     'Name',
     'Eng. Name',
     'Postal Code',
     '縣市別',
     '第三級行政區',
     '學校地址']
    """
    Serial_Number = data['Serial Number']
    ID = data['ID']
    Public_Private = data['Public/Private']
    System = data['System']
    Name = data['Name']
    Eng_Name = data['Eng. Name']
    Postal_Code = data['Postal Code']
    address_zh = data['學校地址']
    
    g = geocoder.here(address_zh,
                       app_id=APP_ID,
                       app_code=APP_CODE)
    result = g.json
    
    Latitude = result['lat']
    Longitude = result['lng']
    Address = result['address']
    City = result['city']
    County = result['county']
    if 'neighborhood' in result:
        Neighborhood = result['neighborhood']
    else:
        Neighborhood = np.nan
    updf = updf.append({
        'Serial Number': Serial_Number,
        'ID': ID,
        'Public/Private': Public_Private,
        'System':System,
        'Name': Name,
        'Eng. Name': Eng_Name,
        'Postal Code': Postal_Code,
        'Latitude':Latitude,
        'Longitude': Longitude,
        'Address': Address,
        'City': City,
        'County': County,
        'Neighborhood': Neighborhood}, ignore_index=True
    )

updf

,Serial Number,ID,Public/Private,System,Name,Eng. Name,Postal Code,Latitude,Longitude,Address,City,County,Neighborhood
0,1,0001,Public,一般大學,國立政治大學,National Chengchi Universerity,11605,24.98741,121.57651,"No. 64, Sec 2, Zhi Nan Rd., Wenshan District, ...",Taipei City,Taipei City,Wenshan District
1,2,0002,Public,一般大學,國立清華大學,National Tsing Hua University,30013,24.79635,120.99686,"No. 101, Sec 2, Guang Fu Rd., East District, H...",Hsinchu City,Hsinchu City,East District
2,3,0003,Public,一般大學,國立臺灣大學,NATIONAL TAIWAN UNIVERSITY,10617,25.01697,121.53369,"No. 1, Sec 4, Luo Si Fu Rd., Daan District, Ta...",Taipei City,Taipei City,Daan District
3,4,0004,Public,一般大學,國立臺灣師範大學,National Taiwan Normal University,10610,25.02643,121.52756,"No. 162, Sec 1, He Ping E. Rd., Daan District,...",Taipei City,Taipei City,Daan District
4,5,0005,Public,一般大學,國立成功大學,National Cheng Kung University,70101,22.99632,120.21953,"No. 1, Da Syue Rd., East District, Tainan City...",Tainan City,Tainan City,East District
...,...,...,...,...,...,...,...,...,...,...,...,...,...
155,156,1R05,Private,宗教研修學院,台神學校財團法人台灣神學研究學院,Taiwan Graduate School of Theology,11149,25.10163,121.54090,"No. 20, Lane 2, Sec 2, Yang De Blvd., Shilin D...",Taipei City,Taipei City,Shilin District
156,157,1R06,Private,宗教研修學院,一貫道崇德學院,i.k.t chong-de school,54552,23.96040,121.00169,"No. 25-8, Li Yu Rd., Wu Gong Vil., Puli Townsh...",Puli Township,Nantou County,Wu Gong Vil.
157,158,1R07,Private,宗教研修學院,台灣基督長老教會南神神學院,Tainan Theological College,70142,22.98829,120.21394,"No. 117, Sec 1, Dong Men Rd., East District, T...",Tainan City,Tainan City,East District
158,159,3002,Public,一般大學,臺北市立大學,University of Taipei,10048,25.03556,121.51357,"No. 1, Ai Guo W. Rd., Zhongzheng District, Tai...",Taipei City,Taipei City,Zhongzheng District


In [106]:
updf['County (zh)'] = pdf['縣市別']

In [107]:
updf.head()

,Serial Number,ID,Public/Private,System,Name,Eng. Name,Postal Code,Latitude,Longitude,Address,City,County,Neighborhood,County (zh)
0,1,0001,Public,一般大學,國立政治大學,National Chengchi Universerity,11605,24.98741,121.57651,"No. 64, Sec 2, Zhi Nan Rd., Wenshan District, ...",Taipei City,Taipei City,Wenshan District,臺北市
1,2,0002,Public,一般大學,國立清華大學,National Tsing Hua University,30013,24.79635,120.99686,"No. 101, Sec 2, Guang Fu Rd., East District, H...",Hsinchu City,Hsinchu City,East District,新竹市
2,3,0003,Public,一般大學,國立臺灣大學,NATIONAL TAIWAN UNIVERSITY,10617,25.01697,121.53369,"No. 1, Sec 4, Luo Si Fu Rd., Daan District, Ta...",Taipei City,Taipei City,Daan District,臺北市
3,4,0004,Public,一般大學,國立臺灣師範大學,National Taiwan Normal University,10610,25.02643,121.52756,"No. 162, Sec 1, He Ping E. Rd., Daan District,...",Taipei City,Taipei City,Daan District,臺北市
4,5,0005,Public,一般大學,國立成功大學,National Cheng Kung University,70101,22.99632,120.21953,"No. 1, Da Syue Rd., East District, Tainan City...",Tainan City,Tainan City,East District,臺南市


In [111]:
updf.columns.values

array(['Serial Number', 'ID', 'Public/Private', 'System', 'Name',
       'Eng. Name', 'Postal Code', 'Latitude', 'Longitude', 'Address',
       'City', 'County', 'Neighborhood', 'County (zh)'], dtype=object)

In [109]:
updf.to_csv('p_ulist_2021-02-18.csv', index=False)

In [110]:
df = pd.read_csv('p_ulist_2021-02-18.csv')
df.head()

,Serial Number,ID,Public/Private,System,Name,Eng. Name,Postal Code,Latitude,Longitude,Address,City,County,Neighborhood,County (zh)
0,1,0001,Public,一般大學,國立政治大學,National Chengchi Universerity,11605,24.98741,121.57651,"No. 64, Sec 2, Zhi Nan Rd., Wenshan District, ...",Taipei City,Taipei City,Wenshan District,臺北市
1,2,0002,Public,一般大學,國立清華大學,National Tsing Hua University,30013,24.79635,120.99686,"No. 101, Sec 2, Guang Fu Rd., East District, H...",Hsinchu City,Hsinchu City,East District,新竹市
2,3,0003,Public,一般大學,國立臺灣大學,NATIONAL TAIWAN UNIVERSITY,10617,25.01697,121.53369,"No. 1, Sec 4, Luo Si Fu Rd., Daan District, Ta...",Taipei City,Taipei City,Daan District,臺北市
3,4,0004,Public,一般大學,國立臺灣師範大學,National Taiwan Normal University,10610,25.02643,121.52756,"No. 162, Sec 1, He Ping E. Rd., Daan District,...",Taipei City,Taipei City,Daan District,臺北市
4,5,0005,Public,一般大學,國立成功大學,National Cheng Kung University,70101,22.99632,120.21953,"No. 1, Da Syue Rd., East District, Tainan City...",Tainan City,Tainan City,East District,臺南市


In [105]:
df[df['Name'] == '景文科技大學']

,Serial Number,ID,Public/Private,System,Name,Eng. Name,Postal Code,Latitude,Longitude,Address,City,County,Neighborhood
101,102,1054,Private,技專校院,景文科技大學,Jinwen University of Science and Technology,23154,24.95199,121.50928,"No. 99, An Zhong Rd., Xindian District, New Ta...",New Taipei City,New Taipei City,Xindian District


In [85]:
g = geocoder.here('台中市北屯區廍子路666號',
                       app_id=APP_ID,
                       app_code=APP_CODE)

In [86]:
g.json

{'address': 'No. 660, Buzi Rd., Beitun District, Taichung City, 406, Taiwan',
 'bbox': {'northeast': [24.1713448, 120.7347696],
  'southwest': [24.1690965, 120.7323052]},
 'city': 'Taichung City',
 'confidence': 9,
 'country': 'TWN',
 'county': 'Taichung City',
 'district': 'Beitun District',
 'housenumber': '660',
 'lat': 24.1702206,
 'lng': 120.7335374,
 'neighborhood': 'Beitun District',
 'ok': True,
 'postal': '406',
 'raw': {'LocationId': 'NT_KGdwth6gOWa-LHY3dxZegA_2YDM',
  'LocationType': 'address',
  'DisplayPosition': {'Latitude': 24.1702206, 'Longitude': 120.7335374},
  'NavigationPosition': [{'Latitude': 24.1702855, 'Longitude': 120.7334077}],
  'MapView': {'TopLeft': {'Latitude': 24.1713448, 'Longitude': 120.7323052},
   'BottomRight': {'Latitude': 24.1690965, 'Longitude': 120.7347696}},
  'Address': {'Label': 'No. 660, Buzi Rd., Beitun District, Taichung City, 406, Taiwan',
   'Country': 'TWN',
   'County': 'Taichung City',
   'City': 'Taichung City',
   'District': 'Beitun

In [64]:
json_sample_data['address']

'No. 64, Sec 2, Zhi Nan Rd., Wenshan District, Taipei City, 116, Taiwan'

In [22]:
json_sample_data = {'address': 'No. 64, Sec 2, Zhi Nan Rd., Wenshan District, Taipei City, 116, Taiwan', 'bbox': {'northeast': [24.9885342, 121.5777502], 'southwest': [24.9862858, 121.5752698]}, 'city': 'Taipei City', 'confidence': 9, 'country': 'TWN', 'county': 'Taipei City', 'district': 'Wenshan District', 'housenumber': '64', 'lat': 24.98741, 'lng': 121.57651, 'neighborhood': 'Wenshan District', 'ok': True, 'postal': '116', 'raw': {'LocationId': 'NT_PoyY44y4nuC5D7axQNc9AD_2QD', 'LocationType': 'address', 'DisplayPosition': {'Latitude': 24.98741, 'Longitude': 121.57651}, 'NavigationPosition': [{'Latitude': 24.98751, 'Longitude': 121.57653}], 'MapView': {'TopLeft': {'Latitude': 24.9885342, 'Longitude': 121.5752698}, 'BottomRight': {'Latitude': 24.9862858, 'Longitude': 121.5777502}}, 'Address': {'Label': 'No. 64, Sec 2, Zhi Nan Rd., Wenshan District, Taipei City, 116, Taiwan', 'Country': 'TWN', 'County': 'Taipei City', 'City': 'Taipei City', 'District': 'Wenshan District', 'Street': 'Sec 2, Zhi Nan Rd.', 'HouseNumber': '64', 'PostalCode': '116', 'AdditionalData': [{'value': 'Taiwan', 'key': 'CountryName'}, {'value': 'Taipei City', 'key': 'CountyName'}]}, 'CountryName': 'Taiwan', 'CountyName': 'Taipei City'}, 'status': 'OK', 'street': 'Sec 2, Zhi Nan Rd.'}
jsd = pd.json_normalize(json_sample_data)
jsd.columns.to_list()

['address',
 'city',
 'confidence',
 'country',
 'county',
 'district',
 'housenumber',
 'lat',
 'lng',
 'neighborhood',
 'ok',
 'postal',
 'status',
 'street',
 'bbox.northeast',
 'bbox.southwest',
 'raw.LocationId',
 'raw.LocationType',
 'raw.DisplayPosition.Latitude',
 'raw.DisplayPosition.Longitude',
 'raw.NavigationPosition',
 'raw.MapView.TopLeft.Latitude',
 'raw.MapView.TopLeft.Longitude',
 'raw.MapView.BottomRight.Latitude',
 'raw.MapView.BottomRight.Longitude',
 'raw.Address.Label',
 'raw.Address.Country',
 'raw.Address.County',
 'raw.Address.City',
 'raw.Address.District',
 'raw.Address.Street',
 'raw.Address.HouseNumber',
 'raw.Address.PostalCode',
 'raw.Address.AdditionalData',
 'raw.CountryName',
 'raw.CountyName']

In [79]:
import numpy as np

In [81]:
'neighborhood' in json_sample_data
np.nan

nan

In [4]:
# define the dataframe columns
column_names = ['Serial Number',
 'ID',
 'Public/Private',
 'System',
 'Name',
 'Eng. Name',
 'Postal Code', 
 'Latitude', 
 'Longitude', 
 'Address',
 'City',
 'County',
 'Neighborhood'] 

# instantiate the dataframe
updf = pd.DataFrame(columns=column_names)
updf

,Serial Number,ID,Public/Private,System,Name,Eng. Name,Postal Code,Latitude,Longitude,Address,City,County,Neighborhood


In [4]:
# define the dataframe columns
column_names = ['Serial Number',
 'ID',
 'Public/Private',
 'System',
 'Name',
 'Eng. Name',
 'Postal Code', 
 'Latitude', 
 'Longitude', 
 'Address',
 'City',
 'County',
 'Neighborhood'] 

# instantiate the dataframe
updf = pd.DataFrame(columns=column_names)
updf

,Serial Number,ID,Public/Private,System,Name,Eng. Name,Postal Code,Latitude,Longitude,Address,City,County,Neighborhood


In [8]:
for data in tqdm(pdf.to_dict(orient='records')):
    """
    ['Serial Number',
     'ID',
     'Public/Private',
     'System',
     'Name',
     'Eng. Name',
     'Postal Code',
     '縣市別',
     '第三級行政區',
     '學校地址']
    """
    Serial_Number = data['Serial Number']
    ID = data['ID']
    Public_Private = data['Public/Private']
    System = data['System']
    Name = data['Name']
    Eng_Name = data['Eng. Name']
    Postal_Code = data['Postal Code']
    address_zh = data['學校地址']
    county_zh = data['縣市別']
    
    g = geocoder.here(address_zh,
                       app_id=APP_ID,
                       app_code=APP_CODE)
    result = g.json
    
    Latitude = result['lat']
    Longitude = result['lng']
    Address = result['address']
    City = result['city']
    County = result['county']
    if 'neighborhood' in result:
        Neighborhood = result['neighborhood']
    else:
        Neighborhood = np.nan
    updf = updf.append({
        'Serial Number': Serial_Number,
        'ID': ID,
        'Public/Private': Public_Private,
        'System':System,
        'Name': Name,
        'Eng. Name': Eng_Name,
        'Postal Code': Postal_Code,
        'Latitude':Latitude,
        'Longitude': Longitude,
        'Address': Address,
        'City': City,
        'County': County,
        'Neighborhood': Neighborhood,
        'County (zh)': county_zh}, ignore_index=True
    )

updf.head()

,Serial Number,ID,Public/Private,System,Name,Eng. Name,Postal Code,Latitude,Longitude,Address,City,County,Neighborhood,County (zh)
0,1,0001,Public,一般大學,國立政治大學,National Chengchi Universerity,11605,24.98741,121.57651,"No. 64, Sec 2, Zhi Nan Rd., Wenshan District, ...",Taipei City,Taipei City,Wenshan District,臺北市
1,2,0002,Public,一般大學,國立清華大學,National Tsing Hua University,30013,24.79635,120.99686,"No. 101, Sec 2, Guang Fu Rd., East District, H...",Hsinchu City,Hsinchu City,East District,新竹市
2,3,0003,Public,一般大學,國立臺灣大學,NATIONAL TAIWAN UNIVERSITY,10617,25.01697,121.53369,"No. 1, Sec 4, Luo Si Fu Rd., Daan District, Ta...",Taipei City,Taipei City,Daan District,臺北市
3,4,0004,Public,一般大學,國立臺灣師範大學,National Taiwan Normal University,10610,25.02643,121.52756,"No. 162, Sec 1, He Ping E. Rd., Daan District,...",Taipei City,Taipei City,Daan District,臺北市
4,5,0005,Public,一般大學,國立成功大學,National Cheng Kung University,70101,22.99632,120.21953,"No. 1, Da Syue Rd., East District, Tainan City...",Tainan City,Tainan City,East District,臺南市
